# ENIGH Data

This notebook extracts and cleans the data of the Income and Expenditure Survey. Such files can be downloaded from the INEGI's website with the following link:
https://www.inegi.org.mx/programas/enigh/nc/2018/default.html#Microdatos

In [1]:
import pandas as pd
import geopandas as gpd
from functools import reduce

In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
def extractENIGH(dataRoot, year, urban=True):
    ''' 
    Extracts data from the ENIGH tables, these must be previously downloaded and stored in a folder with
    the folder name equal to the corresponding year of the ENIGH data (Example: "Data\2018\hogar.csv"). The 
    datapoints (households) are merged with the metropolitan area where they are located (by default: inner merge,
    urban=False, left merge). All the data taken from the tables is returned as a single dataFrame.
    
    -dataRoot: root folder where the data is located
    -year: year and name folder where the data is extracted (dataRoot\year\example.csv)
    -urban: if True the dataFrame only includes urban households, if false, returns all households.
    '''
        
    # file path
    tabConHogar = r'/Ingresos y Gastos de los Hogares/' + year + '/concentradohogar.csv'
    tabHogar = r'/Ingresos y Gastos de los Hogares/' + year + '/hogares.csv'
    tabPoblacion = r'\Ingresos y Gastos de los Hogares/' + year + '/poblacion.csv'
    tabViviendas = r'\Ingresos y Gastos de los Hogares/' + year + '/viviendas.csv'
    tabGastosH = r'\Ingresos y Gastos de los Hogares/' + year + '/gastoshogar.csv'
    tabGastosP = r'\Ingresos y Gastos de los Hogares/' + year + '/gastospersona.csv'
    
    if year=='2018':
        concentrador_cols = list(range(17)) + [19, 22, 56,82,92] # columns of interest
    elif year=='2016':
        concentrador_cols = list(range(0,18)) + [20, 23, 57,83,93] # columns of interest
    else:
        raise ValueError('There is no data for the given year')
        
        
    # Read files
    concentrador = pd.read_csv(dataRoot + tabConHogar, usecols=concentrador_cols, na_values=' ') # read tabla concentrador hogar
    if urban:
        concentrador = concentrador.loc[concentrador['tam_loc']!=4]
    if year == '2016':
        concentrador.insert(3, "CVE_MUN", concentrador.ubica_geo.astype(str).str[:-4].astype(int))
        concentrador.insert(3, "estado", concentrador.CVE_MUN.astype(str).str[:-3].astype(int))
    else:
        concentrador.rename(columns={'ubica_geo':'CVE_MUN'}, inplace=True)
        concentrador.insert(3, "estado", concentrador.CVE_MUN.astype(str).str[:-3].astype(int))
        
    hogares_cols = [0,1]+[39,41,43,45] # columns of interest
    hogares = pd.read_csv(dataRoot + tabHogar, usecols=hogares_cols, na_values=' ') # read tabla poblacion
    hogares['vehiculos'] = hogares.iloc[:,2:].sum(axis=1, min_count=1)
    hogares.drop(columns=['num_auto','num_van','num_pickup','num_moto'], inplace=True)

    poblacion_cols = [0,1,2,5,40] # columns of interest
    poblacion = pd.read_csv(dataRoot + tabPoblacion, usecols=poblacion_cols, na_values=' ') # read tabla poblacion

    vivienda_cols = [0,1,5,10] + list(range(21,25)) + [27,46,47,50,51,52]  # columns of interest
    vivienda = pd.read_csv(dataRoot + tabViviendas, usecols=vivienda_cols, na_values=['&',' '], dtype={'combustible': float}) # read tabla vivienda

    gastosH_cols = [0,1,2,23]
    gastosH = pd.read_csv(dataRoot + tabGastosH, usecols=gastosH_cols, na_values=' ') # read tabla gastoshogar
    gastosH = gastosH.groupby(['folioviv', 'foliohog', 'clave'],as_index=False).sum(min_count=1)

    gastosP_cols = [0,1,3,17]
    gastosP = pd.read_csv(dataRoot + tabGastosP, usecols=gastosP_cols, na_values=' ') # read tabla gastoshogar
    gastosP = gastosP.groupby(['folioviv', 'foliohog', 'clave'],as_index=False).sum(min_count=1)
    
    
    # Add metropolitan area code and municpality name
    ZM_2015 = pd.read_csv(dataRoot + "\Zonas metropolitanas\ZM_2015.csv", encoding='latin-1', usecols=list(range(6)))
    if urban:
        concentrador = concentrador.merge(ZM_2015[['CVE_ZM','NOM_ZM','CVE_ENT','NOM_ENT','CVE_MUN','NOM_MUN']], 
                                      on='CVE_MUN', how='inner')
    else:
        concentrador = concentrador.merge(ZM_2015[['CVE_ZM','NOM_ZM','CVE_ENT','NOM_ENT','CVE_MUN','NOM_MUN']], 
                                      on='CVE_MUN', how='left')
    # fix column order    
    concentrador = concentrador[list(concentrador)[:4]+list(concentrador)[-6:]+list(concentrador)[4:-6]]
    
    
    # Transform values to school years
    dic_edu = {0:0,1:0,2:6,3:9,4:12,5:15,6:15,7:16,8:18,9:21} # for mapping the number of school years equivalent to each value 
    poblacion['nivelaprob'] = poblacion['nivelaprob'].map(dic_edu)

    # Age
    pobEdad = poblacion.groupby(['folioviv','foliohog'], sort=False, as_index=False)['edad'].mean() # Get average age

    # Education
    pobEdu = poblacion.loc[poblacion['edad']>=15].groupby(['folioviv','foliohog'], 
                                sort=False, as_index=False)['nivelaprob'].mean() # Get mean education of population > 15 y
    pobEdu_men15 = poblacion.groupby(['folioviv','foliohog'], 
                                sort=False, as_index=False)['nivelaprob'].mean() # Get mean poblacion_ of whole population to fill houses with no >15 population

    # merge
    pob = pobEdad.merge(pobEdu, on=['folioviv','foliohog'], how='left')
    pob['nivelaprob'] = pob['nivelaprob'].fillna(pobEdu_men15['nivelaprob']) # give a value for households with no population older than 15 years old
    
    # Merge with concentrador Hogar
    con = concentrador.merge(pob, on=['folioviv','foliohog'], how='left')
    
    # Merge with vehicules Hogar
    con = concentrador.merge(hogares, on=['folioviv','foliohog'], how='left')
    
    # Housing
    con = con.merge(vivienda, on='folioviv', how='left') # Merge dataframes

    
    # Expenses' keys
    claves = ['R001','R003','G009','G010','G011','G012','G013','G014']
    dic_cves = {'R001':'ele', 'R003':'gas', 'G009':'lpg', 'G010':'oil', 'G011':'diesel', 'G012':'coal', 
                'G013':'wood', 'G014':'heat', 'F007':'Magna', 'F008':'Premium', 'F009':'Die-Gas'}

    # Sort every type of expenses into columns
    def mergeGastos(gastoDF):
        gastoList = []
        for clave in dic_cves:
            gasto_x = gastoDF.loc[gastoDF['clave']==clave].copy()
            gasto_x.rename(columns={'gasto_tri':'gasto_tri_' + dic_cves[clave]}, inplace=True)
            gasto_x.drop(['clave'], axis=1, inplace=True)
            gastoList.append(gasto_x)

        # merge all the expenses
        return reduce(lambda  left,right: pd.merge(left,right, on=['folioviv','foliohog'], how='outer'), gastoList)

    gastH = mergeGastos(gastosH)
    gastP = mergeGastos(gastosP)

    # Sum expenses from hogar and persona
    gasto = gastH.set_index(['folioviv', 'foliohog']).add(gastP.set_index(['folioviv', 'foliohog']), fill_value=0)
    gasto.reset_index(inplace=True)

    # Merge with concentrador Hogar
    con = con.merge(gasto, on=['folioviv','foliohog'], how='left')
    
    return con

## Data extraction
Give the root folder of the files and the version (year) of the ENIGH survey to analyze

In [6]:
dataRoot = r'D:\Tesis\Datos' #path of data folder
year = '2018'
con_2018 = extractENIGH(dataRoot, year, urban=True)

In [7]:
len(con_2018)

31100

In [19]:
dataRoot = r'D:\Tesis\Datos' #path of data folder
year = '2018'
con_2018 = extractENIGH(dataRoot, year, urban=True)

In [22]:
dataRoot = r'D:\Tesis\Datos' #path of data folder
year = '2016'
con_2016 = extractENIGH(dataRoot, year, urban=True)

In [23]:
con_2016

,folioviv,foliohog,ubica_geo,estado,publico,CVE_ZM,NOM_ZM,CVE_ENT,NOM_ENT,NOM_MUN,CVE_MUN,ageb,tam_loc,est_socio,est_dis,upm,factor,clase_hog,sexo_jefe,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,ocupados,ing_cor,gasto_mon,energia,vehiculos,tipo_viv,antiguedad,num_cuarto,disp_elect,focos_inca,focos_ahor,combustible,tenencia,calent_sol,calent_gas,tanque_gas,aire_acond,calefacc,gasto_tri_ele,gasto_tri_gas,gasto_tri_lpg,gasto_tri_oil,gasto_tri_diesel,gasto_tri_coal,gasto_tri_wood,gasto_tri_heat,gasto_tri_Magna,gasto_tri_Premium,gasto_tri_Die-Gas
0,100003801,1,10010001,1,0.00,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1001,023-3,1,4,3,10,247,2,1,33,10,2,1,1,2,0,2,100696.70,46599.96,1243.5,2,1.0,NaN,5,1,3.0,8.0,3.0,1,2,1,1,2,2,304.5,NaN,894.0,NaN,NaN,NaN,45.0,NaN,2100.0,NaN,NaN
1,100003802,1,10010001,1,0.00,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1001,023-3,1,4,3,10,247,2,1,29,10,2,1,1,2,0,2,146616.16,82427.75,235.5,1,2.0,NaN,5,1,0.0,7.0,3.0,1,2,1,1,2,2,235.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,NaN,NaN
2,100003803,1,10010001,1,0.00,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1001,023-3,1,4,3,10,247,2,1,47,10,6,2,4,3,3,1,94622.95,54792.51,300.0,2,1.0,20.0,8,1,0.0,17.0,3.0,4,1,2,1,2,2,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9000.0,NaN,NaN
3,100003804,1,10010001,1,0.00,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1001,023-3,1,4,3,10,247,3,2,29,11,3,0,3,3,0,2,58278.65,42452.37,3225.0,3,1.0,NaN,5,1,3.0,10.0,3.0,1,2,1,1,2,2,225.0,NaN,3000.0,NaN,NaN,NaN,NaN,NaN,1800.0,NaN,NaN
4,100003805,1,10010001,1,0.00,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1001,023-3,1,4,3,10,247,2,1,55,10,2,2,0,2,0,1,57295.07,47589.29,2445.0,2,1.0,7.0,6,1,0.0,12.0,3.0,4,2,1,1,2,2,345.0,NaN,2100.0,NaN,NaN,NaN,NaN,NaN,900.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30659,3202655820,1,320570001,32,1954.28,32.01,Zacatecas-Guadalupe,32,Zacatecas,Trancoso,32057,011-1,3,2,533,78620,251,2,1,58,4,7,4,3,7,0,5,50456.52,20087.26,2460.0,0,1.0,NaN,4,1,0.0,5.0,3.0,2,2,1,2,2,2,1200.0,NaN,1260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30660,3202655821,1,320570001,32,2815.65,32.01,Zacatecas-Guadalupe,32,Zacatecas,Trancoso,32057,011-1,3,2,533,78620,251,3,1,45,5,5,2,3,5,0,3,41643.42,19669.13,1425.0,1,1.0,20.0,4,1,0.0,5.0,3.0,4,2,2,2,2,2,225.0,NaN,1200.0,NaN,NaN,NaN,NaN,NaN,1200.0,NaN,NaN
30661,3202655822,1,320570001,32,0.00,32.01,Zacatecas-Guadalupe,32,Zacatecas,Trancoso,32057,011-1,3,2,533,78620,251,2,2,55,6,2,1,1,2,0,2,17744.24,9677.25,525.0,1,1.0,20.0,2,1,2.0,0.0,3.0,2,2,2,2,2,2,225.0,NaN,300.0,NaN,NaN,NaN,NaN,NaN,900.0,NaN,NaN
30662,3202655823,1,320570001,32,539.98,32.01,Zacatecas-Guadalupe,32,Zacatecas,Trancoso,32057,011-1,3,2,533,78620,251,2,1,25,4,4,2,2,2,2,1,11644.83,8751.65,1134.0,0,1.0,5.0,2,1,4.0,0.0,3.0,2,2,2,2,2,2,834.0,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Completeness

In [17]:
con_df = con_2018 # name of dataframe

# Columns for completeness check
subset = list(con_df)[list(con_df).index('folioviv'):list(con_df).index('factor')]+['publico'] + \
            list(con_df)[list(con_df).index('disp_elect'):]

In [ ]:
percent_completeness = con_df[subset].notnull().sum() * 100 / len(con_df[subset])
print(percent_completeness,'\n','-'*30)

keyDict = {1:'public', 2:'priv plant', 3:'solar', 4:'other', 5:'no elec'}
s = (con_df['disp_elect'].value_counts(normalize=True, dropna=False) * 100).to_frame()
s['key'] = s.index.to_series().map(keyDict)
print(s,'\n','-'*30)

keyDict = {1:'wood', 2:'coal', 3:'gas tank', 4:'gas pipe', 5:'elec', 6:'other'}
s = (con_df['combustible'].value_counts(normalize=True, dropna=False) * 100).to_frame()
s['key'] = s.index.to_series().map(keyDict)
print(s,'\n','-'*30)

keyDict = {1:'yes', 2:'no'}
s = (con_df['calent_sol'].value_counts(normalize=True, dropna=False) * 100).to_frame()
s['key'] = s.index.to_series().map(keyDict)
print(s,'\n','-'*30)

s = (con_df['calent_gas'].value_counts(normalize=True, dropna=False) * 100).to_frame()
s['key'] = s.index.to_series().map(keyDict)
print(s,'\n','-'*30)

s = (con_df['tanque_gas'].value_counts(normalize=True, dropna=False) * 100).to_frame()
s['key'] = s.index.to_series().map(keyDict)
print(s,'\n','-'*30)

s = (con_df['aire_acond'].value_counts(normalize=True, dropna=False) * 100).to_frame()
s['key'] = s.index.to_series().map(keyDict)
print(s,'\n','-'*30)

s = (con_df['calefacc'].value_counts(normalize=True, dropna=False) * 100).to_frame()
s['key'] = s.index.to_series().map(keyDict)
print(s,'\n','-'*30)

keyDict = {1:'rented', 2:'lend', 3:'own and paying', 4:'own', 5:'litigated', 6:'other'}
s = (con_df['tenencia'].value_counts(normalize=True, dropna=False) * 100).to_frame()
s['key'] = s.index.to_series().map(keyDict)
print(s,'\n','-'*30)

## Save data
Choose the path to save the dataframe in pickle format

In [24]:
pkls_path = pkls = r'D:\Tesis\ResEleCon-MX\pickles'
con_2016.to_pickle(pkls + '\concentrador_2016.pkl')

## Data Metropolis/State

In [19]:
dataRoot = r'D:\Tesis\Datos' #path of data folder
year = '2018'
con_state = extractENIGH(dataRoot, year, urban=False)

In [20]:
ZM_2015 = pd.read_csv(dataRoot + "\Zonas metropolitanas\ZM_2015.csv", encoding='latin-1', usecols=list(range(6)))
zm_mapper = ZM_2015[['CVE_ZM','NOM_ZM']].drop_duplicates().set_index('CVE_ZM').to_dict()['NOM_ZM']
st_mapper = ZM_2015[['CVE_ENT','NOM_ENT']].drop_duplicates().set_index('CVE_ENT').to_dict()['NOM_ENT']

In [21]:
state_list = []
for state in con_state['estado'].unique():
    for metro in con_state.loc[(con_state['estado']==state) & (con_state['CVE_ZM'].notnull()) , 'CVE_ZM'].unique():
        state_list.append([state, st_mapper[state], metro, zm_mapper[metro], 
                          len(con_state[(con_state['CVE_ZM']==metro) & (con_state['tam_loc']!=4)
                                       & (con_state['estado']==state)])/len(con_state[con_state['estado']==state])])
    

In [22]:
stateMetro = pd.DataFrame(state_list, columns =['State cod','StateName','Metropolitan code', 'Metropolitan name','state_ratio']) 
stateMetro

,State cod,StateName,Metropolitan code,Metropolitan name,state_ratio
0,1,Aguascalientes,1.01,Aguascalientes,0.540330
1,2,Baja California,2.01,Ensenada,0.060807
2,2,Baja California,2.02,Mexicali,0.125527
3,2,Baja California,2.03,Tijuana,0.272426
4,3,Baja California Sur,3.01,La Paz,0.258606
5,4,Campeche,4.01,Campeche,0.294956
6,5,Coahuila de Zaragoza,5.02,Monclova-Frontera,0.110169
7,5,Coahuila de Zaragoza,5.03,Piedras Negras,0.055869
8,5,Coahuila de Zaragoza,5.04,Saltillo,0.234463
9,5,Coahuila de Zaragoza,5.01,La Laguna,0.229127


In [160]:
# stateMetro.to_clipboard(index=False)

## Shapefiles Rural

In [68]:
dataRoot = r'D:\Tesis\Datos' #path of data folder
year = '2016'
con_2016 = extractENIGH(dataRoot, year, urban=False)
con_2016['CVEGEO'] = con_2016.ubica_geo.map("{:09}".format) + con_2016.ageb.str.replace('-','')

In [71]:
con_2016.loc[(con_2016.tam_loc==4) & (con_2016.folioviv.astype(str).str[2]!='6') & (con_2016.CVE_ZM.notnull()), 'type'] = '04'
con_2016.loc[(con_2016.tam_loc!=4) & (con_2016.folioviv.astype(str).str[2]=='6') & (con_2016.CVE_ZM.notnull()), 'type'] = '60'
con_2016.loc[(con_2016.tam_loc==4) & (con_2016.folioviv.astype(str).str[2]=='6') & (con_2016.CVE_ZM.notnull()), 'type'] = '64'

rural = con_2016[con_2016.type.notnull()]

In [69]:
ageb_shp = gpd.read_file(r'D:\Tesis\Datos\shapefiles\AGEB\ageb.shp')
ageb_rural = ageb_shp.merge(rural[['CVEGEO', 'type']], on='CVEGEO', how='inner')
ageb_rural.to_file(r'D:\Tesis\Datos\shapefiles\AGEB\ageb_rural.shp')

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry
0,010010001216A,01,1001,0001,216A,"POLYGON ((2472175.611 1095222.905, 2472149.898..."
1,0100100012649,01,1001,0001,2649,"POLYGON ((2474436.993 1094330.964, 2474423.306..."
2,010010001383A,01,1001,0001,383A,"POLYGON ((2472973.672 1091954.736, 2472991.031..."
3,010010001287A,01,1001,0001,287A,"POLYGON ((2468473.209 1093564.706, 2468482.070..."
4,010010001084A,01,1001,0001,084A,"POLYGON ((2469318.936 1094912.739, 2469386.691..."


In [75]:
len(ageb_rural)

915

In [76]:
len(rural)

9375